In [1]:
import numpy as np
import os
import cv2 as cv

from constants import *
from hsv_sliders import *
from image_processing import *
from visualisation import *
from solution import *


In [2]:
# get files 

files = os.listdir(INPUT_DIR)

solutions = sorted([file for file in files if os.path.splitext(file)[1] == ".txt"])
photos = sorted([file for file in files if os.path.splitext(file)[1] == ".jpg"])
turns_files = sorted([file for file in files if file.split("_")[1] == "turns.txt"])

# print(photos)


In [3]:
# show template board and all squares
tpl = cv.imread(TEMPLATE_PATH)

board = get_trimmed(tpl)
board = process_board(board)

# show_squares(board)


In [4]:
# test 2 photos
templates = load_templates()

idx = 164
prev_board = get_board(photos[idx - 1])
board = get_board(photos[idx])

move_pos = get_board_change(prev_board, board)
res_string = str(BOARD_Y_VALS[move_pos[Y]]) + str(BOARD_X_VALS[move_pos[X]])

flt_board = process_board(board)
current_square = process_square(get_square(flt_board, move_pos[X], move_pos[Y]))
# plt.imshow(current_square)
# plt.show()
# plt.imshow(templates[28])
# plt.show()
# plt.imshow(templates[25])
# plt.show()
# print(get_similitude(current_square, templates))

move_val = get_similitude(process_square(get_square(flt_board, move_pos[X], move_pos[Y])), templates)
res_string = res_string + " " + str(move_val)

print(res_string, "\t", end = "\n")



11G 28 	


In [ ]:
# print_train_res()

def get_players_moves(turns_file):
    players_moves = []

    with open(os.path.join(INPUT_DIR, turns_file), "r") as file:
        for line in file.readlines():
            players_moves.append(int(line.split(" ")[1].replace("\n", "")) - 1)

    return players_moves

def init_move_matrix():
    mat = np.full((14, 14), -1)
    mat[6][6] = 1
    mat[6][7] = 2
    mat[7][6] = 3
    mat[7][7] = 4

    return mat

def check_ecuations(a, b, res, constraint):
    if constraint == NC:
        if a + b == res or abs(a - b) == res or a * b == res:
            return True
        if a != 0 and b != 0 and max(a, b) / min(a, b) == res:
            return True

    if constraint == PL and a + b == res:
        return True
    if constraint == MI and abs(a - b) == res:
        return True
    if constraint == MU and a * b == res:
        return True
    if constraint == DV: 
        if a != 0 and b != 0 and max(a, b) / min(a, b) == res:
            return True
    
    return False


def get_score(x, y, val, board_mat):
    ecuations_found = 0
    # sus:
    if y > 1:
        a = board_mat[y - 1][x] 
        b = board_mat[y - 2][x]
        if a != -1 and b != -1:
            if check_ecuations(a, b, val, CONSTRAINTS[y][x]):
                ecuations_found += 1
    if y < 12:
        a = board_mat[y + 1][x] 
        b = board_mat[y + 2][x]
        if a != -1 and b != -1: 
            if check_ecuations(a, b, val, CONSTRAINTS[y][x]):
                ecuations_found += 1
    if x > 1:
        a = board_mat[y][x - 1] 
        b = board_mat[y][x - 2]
        if a != -1 and b != -1:
            if check_ecuations(a, b, val, CONSTRAINTS[y][x]):
                ecuations_found += 1
    if x < 12:
        a = board_mat[y][x + 1] 
        b = board_mat[y][x + 2]
        if a != -1 and b != -1:
            if check_ecuations(a, b, val, CONSTRAINTS[y][x]):
                ecuations_found += 1

    return val * ecuations_found * BONUSES[y][x]



In [45]:
data_set = -1
templates = load_templates()
moves_results = []

for photo in photos: 
    current_set = int(photo.split('_')[0])
    if current_set != data_set:
        moves_results.append(move_score)
        print(moves_results)
        if data_set != -1:
            print_mat(move_matrix)
        prev_board = get_board(EMPTY_BOARD_PATH)
        move_matrix = init_move_matrix()
        data_set = current_set

        turns_file = [file for file in turns_files if int(file.split("_")[0]) == data_set][0]
        print(turns_file)
        players_moves = get_players_moves(turns_file)
        players_moves.append(1000)
        print(players_moves)

        moves_results = []
        current_move = 0
        current_turn_idx = 0
        move_score = 0

    # print(current_move)
    # print(players_moves[current_turn_idx + 1])

    if current_move >= players_moves[current_turn_idx + 1]:
        print()
        moves_results.append(move_score)
        current_turn_idx += 1
        move_score = 0

    # photo = photos[49]
    board = get_board(photo)
    move_pos = get_board_change(prev_board, board)

    res_string = str(BOARD_Y_VALS[move_pos[Y]]) + str(BOARD_X_VALS[move_pos[X]])
    move_val = get_similitude(process_square(get_square(process_board(board), move_pos[X], move_pos[Y])), templates)
    res_string = res_string + " " + str(move_val)

    with open(os.path.join(OUTPUT_DIR, os.path.splitext(photo)[0] + ".txt"), 'w') as file:
        file.write(res_string)

    move_matrix[move_pos[Y], move_pos[X]] = move_val
    move_score += get_score(move_pos[X], move_pos[Y], move_val, move_matrix)
    prev_board = board
    current_move += 1 

    print(move_score, ", ", end=" ")

moves_results.append(move_score)
print(moves_results)
print_mat(move_matrix)

# templates = load_templates()

# show_squares(flt_board)
# show_img(flt_board)

# hsv_sliders(flt_board, RESIZE_SHOW, RESIZE_SHOW)

# 1 0 1 1 1 0 0 1 1 1 1 1 0 1 
# 1 1 1 1 1 1 1 1 1 1 1 1 0 1
# 152 170 28 35 16 30 128 126 111 13 10 147 38 13

[36]
1_turns.txt
[0, 4, 10, 13, 17, 20, 23, 27, 30, 33, 35, 38, 42, 47, 1000]
8 ,  40 ,  80 ,  152 ,  
8 ,  18 ,  98 ,  106 ,  116 ,  170 ,  
3 ,  14 ,  28 ,  
5 ,  8 ,  23 ,  35 ,  
3 ,  10 ,  16 ,  
18 ,  27 ,  30 ,  
3 ,  13 ,  30 ,  128 ,  
9 ,  72 ,  126 ,  
36 ,  84 ,  111 ,  
6 ,  13 ,  
3 ,  5 ,  10 ,  
7 ,  17 ,  87 ,  147 ,  
4 ,  9 ,  19 ,  22 ,  38 ,  
5 ,  13 ,  13 ,  [152, 170, 28, 35, 16, 30, 128, 126, 111, 13, 10, 147, 38, 13]
_	_	_	_	_	_	_	18	_	5	_	_	_	_	
_	_	_	_	_	_	_	10	_	2	_	_	_	_	
_	_	_	_	_	_	_	8	_	3	_	_	_	_	
_	_	_	_	_	_	_	80	_	6	_	48	_	_	
_	_	_	_	_	_	_	10	_	9	_	36	_	_	
_	_	_	3	14	11	3	8	5	3	15	12	27	_	
_	_	_	_	_	_	1	2	3	6	9	3	_	_	
_	_	_	_	_	7	3	4	7	9	63	_	_	_	
_	_	_	_	8	10	2	8	10	_	54	_	_	_	
_	_	_	5	8	3	5	32	17	49	_	_	_	_	
_	_	_	_	0	_	_	40	7	_	_	_	_	_	
_	_	_	_	_	_	_	72	10	_	_	_	_	_	
_	_	_	_	_	_	_	_	70	_	_	_	_	_	
_	_	_	_	_	_	_	_	60	_	_	_	_	_	

2_turns.txt
[0, 6, 12, 15, 18, 22, 28, 32, 38, 40, 43, 46, 1000]
3 ,  9 ,  18 ,  33 ,  39 ,  309 ,  
3 ,  10 ,  31 ,  37 , 

/tmp/ipykernel_4441/2747335727.py:23: RuntimeWarning: divide by zero encountered in scalar divide
  if a + b == res or abs(a - b) == res or max(a, b) / min(a, b) == res or a * b == res:


6 ,  14 ,  62 ,  
49 ,  50 ,  
7 ,  17 ,  34 ,  36 ,  44 ,  54 ,  56 ,  
2 ,  5 ,  16 ,  72 ,  79 ,  124 ,  
10 ,  20 ,  24 ,  45 ,  
25 ,  60 ,  70 ,  86 ,  
3 ,  7 ,  67 ,  73 ,  78 ,  128 ,  [2, 23, 106, 2, 172, 19, 62, 50, 56, 124, 45, 86, 128]
_	_	_	_	_	_	_	_	_	_	_	_	_	_	
_	_	_	_	_	_	_	_	_	_	_	_	_	_	
_	_	_	_	_	_	_	_	_	10	_	_	_	_	
_	_	_	_	_	1	49	48	1	80	81	_	_	_	
_	_	6	_	_	_	_	8	1	8	_	_	_	_	
_	_	10	35	25	10	4	6	0	10	8	_	_	_	
20	4	16	12	4	3	1	2	1	2	3	_	_	_	
_	_	_	_	21	7	3	4	1	5	5	_	_	_	
_	_	_	_	_	10	_	2	2	_	_	_	_	_	
_	_	_	45	28	17	11	8	3	_	_	_	_	_	
_	_	_	_	_	7	_	10	5	50	_	_	_	_	
_	_	_	_	_	10	_	2	_	_	_	_	_	_	
_	_	_	_	_	3	_	_	_	_	_	_	_	_	
_	_	_	_	_	_	_	_	_	_	_	_	_	_	

4_turns.txt
[0, 4, 10, 16, 21, 27, 34, 37, 40, 43, 46, 1000]
3 ,  9 ,  15 ,  23 ,  
4 ,  5 ,  10 ,  19 ,  24 ,  72 ,  
40 ,  48 ,  144 ,  151 ,  179 ,  200 ,  
7 ,  16 ,  22 ,  46 ,  76 ,  
54 ,  99 ,  124 ,  129 ,  144 ,  147 ,  
12 ,  57 ,  58 ,  70 ,  81 ,  91 ,  107 ,  
17 ,  37 ,  47 ,  
14 ,  20 ,  24 ,  
6 ,  66 ,